#### Create CO2 emission data from pre-processed simulation results
- Simulation results should be pre-processed using the latest versions of ```functions_topdown``` or ```functions_bottomup```
- Simulation results should be saved as xlsx-files to the corresponding ```results``` folder

The following code retrieves the xlsx-files and visualizes the data

In [19]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
#plt.rc('figure', facecolor='w')
from IPython.core.pylabtools import figsize
from icecream import ic
import seaborn as sns
sns.set_style('white')
%load_ext autoreload
%autoreload 2
%matplotlib inline
home_path = 'C:\\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\\07_Arbeitsordner\\04_Ergebnisse\\20220920_Ergebnisse_4_Wochen\\new_results_2023'
os.chdir(home_path)
out_path = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\new_results_2023\results\Abbildungen/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
def read_in_files(paths):
    uc_dict = {}
    paths = [paths] if type(paths) != list else paths
    for i in paths:
        uc = i.split('\\')[-1].split('use_case_')[-1].split('.xlsx')[0]
        print(f'reading in use case {uc}')
        if '_v' in uc:
           uc = uc.split('_v')[0]
        #uc = int(i.split('\\')[-1].split('use_case_')[-1][0])
        df = pd.read_excel(i, index_col=0)
        df.index = pd.to_datetime(df.index, format='%Y-%m-%dT%H:%M')
        if len(df) > 672:
            df = df.iloc[:672 - len(df),:]
        uc_dict[uc] = df
    return uc_dict

def filter_cols(df, entity):
    entity = 'grid' if 'germany' in entity and any('grid' in col for col in df.columns.tolist()) else entity
    df_out = df[[f'{entity}_grey_energy [kWh]', f'{entity}_grey_energy [%]']]
    return df_out

def insert_co2_col(df):
    # emission factors based on calculation in: https://fimrc.sharepoint.com/:x:/t/paper-VIdES/EZnJhlevmuVMi3knXpjlRuIBcDw-HWPnGKyYyXgRi7xe6g?e=7Q4Xhm
    gCO2e_kWh_green, gCO2e_kWh_grey = 0.0, 181.79
    entity = df.columns.to_list()[0].split('_')[0]
    df[f'{entity}_CO2e [t]'] = np.where(df[f'{entity}_grey_energy [kWh]'] >= 0,
                                             df[f'{entity}_grey_energy [kWh]'] * gCO2e_kWh_grey / 1e+6,
                                             df[f'{entity}_grey_energy [kWh]'] * (-1) * gCO2e_kWh_green / 1e+6)
    if 'grid' in entity:
        df.columns = [s.replace('grid', 'germany') for s in df.columns.tolist() if 'grid' in s]
    return df

In [21]:
# read in all result data frames
paths = []
for root, dir, file in os.walk(top=home_path+'/results', topdown=True):
    paths += [os.path.join(root, f) for f in file if '.xlsx' in f if 'topdown' in f]

# show files
print([i.split('\\')[-1] for i in paths])

['use_case_2_topdown.xlsx', 'use_case_3_topdown.xlsx', 'use_case_4_topdown.xlsx', 'use_case_6_topdown.xlsx']


In [22]:
# read in all .xslx files as pd.DataFrame
uc_dict = read_in_files(paths[1])

# show results
print(uc_dict.keys())

reading in use case 3_topdown
dict_keys(['3_topdown'])


In [23]:
# for a key in topdown uc_dict, return sum of tCO2 for Germany
def get_sum_co2_germany(key):
    df = uc_dict[key]
    df = filter_cols(df, 'germany')
    df = insert_co2_col(df)
    return df[f'germany_CO2e [t]'].sum(axis=0)

# calculate sum of tCO2 for Germany for all topdown use cases
topdown_co2_germany_results = {}
for key in uc_dict.keys():
    topdown_co2_germany_results[key] = get_sum_co2_germany(key)
    
# print results
print(topdown_co2_germany_results)

{'3_topdown': 44.3961170999638}


C:\Users\joche\AppData\Local\Temp\ipykernel_15036\1668262290.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{entity}_CO2e [t]'] = np.where(df[f'{entity}_grey_energy [kWh]'] >= 0,
